# Automação Web

## Desafio

Trabalhamos em ima importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro
Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.


Para isso, vamos criar uma automação web:
- Usaremos o Selenium
    - **Importante:** Baixar o Webdrive

### Objetivos
- Capturar a cotação do Dólar
- Capturar a cotação do Euro
- Capturar a cotação do Ouro
- Importar a base de dados
- Atualizar a cotação, o preço de compra e o preço de vendas
- Exportar o relatório atualizado


## Preparando o Sistema
- Baixar o Webdriver
    - Google Chrome -> chromedriver
    - Mozila Firefox -> geckodriver
    - colocar na mesma pasta do código fonte
- Importar bibliotecas
- Criar o navegador

In [49]:
#Importando o Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Criando navegador
navegador = webdriver.Chrome('chromedriver')

### Cotação do Dólar

In [50]:
#Acessando o site do Google e acessando xpath
#Cotação Dólar
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cot_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_dolar)

5.5181


### Cotação do Euro

In [51]:
#Cotação Euro
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cot_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_euro)

6.373353292


### Cotação do Ouro

In [52]:
#Cotação Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cot_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value').replace(',','.')
print(cot_ouro)

#Fechando o navegador
navegador.quit()

311.45


## Atualizando a base de dados

In [53]:
import pandas as pd

df_produtos = pd.read_excel('datas/3_produtos.xlsx')
display(df_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [54]:
#Atualizando a Cotação
df_produtos.loc[df_produtos['Moeda'] == 'Dólar' , 'Cotação'] = float(cot_dolar)
df_produtos.loc[df_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cot_euro)
df_produtos.loc[df_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cot_ouro)

display(df_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.518100,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.373353,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.518100,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.518100,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.373353,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.518100,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,311.450000,7000.00,1.15,8050.000


In [55]:
#Atualizar o preço de compra
df_produtos['Preço Base Reais'] = df_produtos['Preço Base Original'] * df_produtos['Cotação']

#Atualizar preço de vendas
df_produtos['Preço Final'] = df_produtos['Preço Base Reais'] * df_produtos['Margem']

#Formatando o preço de venda
#df_produtos['Preço Final'] = df_produtos['Preço Final'].map('R${:.2f}'.format)

display(df_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.518100,5518.044819,1.40,7725.262747
1,Carro Renault,4500.00,Euro,6.373353,28680.089814,2.00,57360.179628
2,Notebook Dell,899.99,Dólar,5.518100,4966.234819,1.70,8442.599192
3,IPhone,799.00,Dólar,5.518100,4408.961900,1.70,7495.235230
4,Carro Fiat,3000.00,Euro,6.373353,19120.059876,1.90,36328.113764
5,Celular Xiaomi,480.48,Dólar,5.518100,2651.336688,2.00,5302.673376
6,Joia 20g,20.00,Ouro,311.450000,6229.000000,1.15,7163.350000


In [56]:
#Exportando a tabela
from datetime import datetime
data = datetime.today().strftime('%Y-%m-%d_%H-%M')
df_produtos.to_excel(f'datas/3_produtos_{data}.xlsx', index=False)